<a href="https://colab.research.google.com/github/MrFzovpec/mettre-marketing/blob/master/mettre-marketing/marketing_analysis/instagram/Instagramm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install allennlp
!pip install transformers

# Organizing the data

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/MrFzovpec/mettre-marketing/master/marketing_analysis/instagram/instagram.csv')

In [0]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 
                 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1'])

In [0]:
df = df.dropna()

In [0]:
df = df.drop_duplicates()

In [0]:
df['index_account'] = None

The accounts are depersonalized, so we need to indexate them

In [0]:
index_acc = 0

In [0]:
for x in df.iterrows():
  if x[0] == 0:
    df['index_account'][x[0]] = index_acc
    previous = x[1]['account_description']
    continue
  if x[1]['account_description'] == previous:
    df['index_account'][x[0]] = index_acc
  else:
    index_acc += 1
    df['index_account'][x[0]] = index_acc

  previous = x[1]['account_description']

# Getting a vocabulary file for the Instagram

In [0]:
from typing import Iterator, List, Dict

In [0]:
from transformers import DistilBertTokenizer

In [0]:
from allennlp.data.tokenizers import Token, Tokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.fields import TextField, SequenceLabelField, LabelField
from allennlp.data import Instance
from allennlp.data.iterators import BucketIterator

In [0]:
class InstagramTextData(DatasetReader):
  def __init__(self, df, token_indexers: Dict[str, TokenIndexer] = None) -> None:
    super().__init__(lazy=False)
    self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
    self.df = df
    # String tokenizer
    self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

  def text_to_instance(self, string: List[Token]) -> Instance:
    sentence_field = TextField(string, self.token_indexers)

    fields = {"string": sentence_field}
    return Instance(fields)

  def _read(self) -> Iterator[Instance]:
      
    for i, row in df.iterrows():
      yield self.text_to_instance(
                [Token('<CLS>') + Token(x) for x in row['text'] + ' ' + row['account_description']],
            )

In [40]:
tok.encode('Hello how r u')

[101, 31178, 14796, 186, 189, 102]

# Creating a dataloader

This is going to be an LSTM model and it's going to work with the five previous examples. So, for example I wanna predict likes countable for some particular post, then I'm going to take 4 previous posts and make a prediction basing on their data. <br> <br>
Here I'll create a few classes which would provide that functionality

In [0]:
class DatasetSamples:
  ''' This class will return a dataset samples in the format of 5 posts 
  (more or less). It's going to be kind of a sliding window'''
  def __init__(self, df, window_size=5):
    self.df = df
    self.window_size = window_size

  def get_window_of_posts(self):
    users = df['index_account'].unique()

    for user in users:
      user = int(user)
      user_df = df[df['index_account'] == user]
      user_df_len = len(user_df)
      user_posts_array = []

      # Identifying the indexes we're going to use to parse
      starts_index = 0
      final_index = user_df_len - self.window_size

      for index in range(starts_index, final_index):
        user_posts_array.append(user_df.iloc[index: index + self.window_size])

      yield user_posts_array

In [0]:
ds = DatasetSamples(df)

In [0]:
d = ds.get_window_of_posts()